Импорт библиотек

In [109]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 1 Разработка модели машинного обучения

Прочитаем наши данные, так как их слишком много, то возьмем первые 100_000 для обучения и 10_000 для валидации 

In [116]:
df=pd.read_csv('./data_X.csv', nrows=1000_000, index_col=0)
target=pd.read_csv('./data_Y.csv', nrows=1000_000, index_col=0)

df_val=pd.read_csv('./data_X_val.csv', nrows=1000_000, index_col=0)
target_val=pd.read_csv('./data_Y_val.csv', nrows=1000_000, index_col=0)

Разобьем данные на обучающую и тестовую выборки в отношении 80% к 20%, что считается устоявшимся и общепринятым в машинном обучении

In [117]:
X_train, X_test, y_train, y_test = train_test_split(df , target, test_size=0.2, random_state=69)

In [ ]:
trans = MinMaxScaler()
dataset = DataFrame( trans.fit_transform(df))

Создадим главную функцию для проверки метрик

In [112]:
def test_metrics(yt,yp):
    print("R2 =",r2_score(yt,yp))
    print("RMSE = ",root_mean_squared_error(yt,yp))


Для обучения буду использовать линейную регрессию - <b>lr</b>, рандомный лес - <b>rfr</b>  и  XGбустинг - <b>xgbr</b>

- Обучаем и проверяем линеную регрессию

In [113]:
lr = LinearRegression()
start_time = time.time()
lr.fit(X_train, y_train)
end_time = time.time()
print('Elapsed time: ', end_time - start_time)
test_metrics(y_test,lr.predict(X_test))

Elapsed time:  8.180866718292236
R2 = 0.34406478160630216
RMSE =  956.8005113236185


In [114]:
pickle.dump(lr, open("lr_reg.pkl", "wb"))

In [119]:
rfr = RandomForestRegressor(max_depth=8)
start_time = time.time()
rfr.fit(X_train, y_train)
end_time = time.time()
print('Elapsed time: ', end_time - start_time)
test_metrics(y_test,rfr.predict(X_test))

/home/c10/main_venv/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Elapsed time:  205.76358580589294
R2 = 0.6703449642130168
RMSE =  679.3972755150041


- Обучаем и проверяем рандомный лес

In [120]:
pickle.dump(rfr, open("rfr_reg.pkl", "wb"))

- Обучаем и проверяем бустинг

In [28]:
xgbr=xgb.XGBRegressor(eval_metric='rmsle',random_state = 69)
start_time = time.time()
xgbr.fit(X_train, y_train)
end_time = time.time()
print('Elapsed time: ', end_time - start_time)
test_metrics(y_test,xgbr.predict(X_test))

Elapsed time:  0.42096805572509766
R2 = 0.8467293977737427
RMSE =  387.08941650390625


Лучшим по соотношению качества и времени оказался бустинг, его и будем использовать

In [31]:
test_metrics(xgbr.predict(df_val),target_val)

R2 = 0.786888837814331
RMSE =  388.37744140625


Реализация функции пайплайна

In [121]:
def pipeline(directory):
    df=pd.read_csv(directory+'data_X.csv', nrows=30_000_000,index_col=0)
    target=pd.read_csv(directory+'data_Y.csv', nrows=30_000_000,index_col=0)

    df_val=pd.read_csv(directory+'data_X_val.csv', nrows=10_000_000,index_col=0)
    target_val=pd.read_csv(directory+'data_Y_val.csv', nrows=10_000_000, index_col=0)

    X_train, X_test, y_train, y_test = train_test_split(df , target, test_size=0.2, random_state=69)
    
    xgbr=xgb.XGBRegressor(eval_metric='rmsle',random_state = 69)
    xgbr.fit(X_train, y_train)

    return xgbr, {'R2':r2_score(xgbr.predict(df_val),target_val),'RMSE':root_mean_squared_error(xgbr.predict(df_val),target_val)}
    

In [122]:
model,data=pipeline('./')

In [123]:
data

{'R2': 0.5808756947517395, 'RMSE': 642.4022827148438}

In [124]:
import pickle
file_name = "xgb_reg.pkl"

# save
pickle.dump(model, open(file_name, "wb"))

# load
xgb_model_loaded = pickle.load(open(file_name, "rb"))

In [70]:
data

{'R2': -120.25090026855469, 'RMSE': 305332.8125}

Функция пайпалайна работает

In [67]:
data

{'R2': -20.860586166381836, 'RMSE': 248390.46875}

In [125]:
df_val_test=pd.read_csv('./data_X_val.csv', nrows=10, index_col=0 )


In [103]:
df_val_test.head()

,0,1,2,3,4,5,7,8,9,10,...,18,19,20,21,22,23,25,26,27,29
0,5,-2,2,0,0,4,0,0,0,-3,...,0,0,4,54562568,0,3,30,-24,4,-24
1,394,0,1,0,0,2,0,0,0,7,...,0,0,6,4747,0,5,23,-7,-28,0
2,776,0,1,0,0,1,0,0,0,4,...,-4,0,10,5028991,0,13,81,-16,0,6
3,594,-4,1,0,0,4,0,0,0,0,...,-1,0,11,29,0,7,161,-20,0,-32
4,196,0,1,0,0,1,0,0,0,3,...,-1,0,4,27311095,0,10,35,5,0,-19


In [126]:
df_val_test.to_csv('test.csv')